<a href="https://colab.research.google.com/github/ibshafique/mlops_with_poridhi/blob/main/prerequisite_projects/WQU_DS/01_housing_in_mexico/mexico_housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisites

## Importing Relevant Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

## Importing Datasets

In [ ]:
url_1 = "https://raw.githubusercontent.com/ibshafique/mlops_with_poridhi/refs/heads/main/prerequisite_projects/WQU_DS/01_housing_in_mexico/dataset/mexico-real-estate-1.csv"
url_2 = "https://raw.githubusercontent.com/ibshafique/mlops_with_poridhi/refs/heads/main/prerequisite_projects/WQU_DS/01_housing_in_mexico/dataset/mexico-real-estate-2.csv"
url_3 = "https://raw.githubusercontent.com/ibshafique/mlops_with_poridhi/refs/heads/main/prerequisite_projects/WQU_DS/01_housing_in_mexico/dataset/mexico-real-estate-3.csv"

In [ ]:
df1 = pd.read_csv(url_1)
df2 = pd.read_csv(url_2)
df3 = pd.read_csv(url_3)

# Data Insights

## Dataset 1

In [ ]:
df1.head()

In [ ]:
df1.shape

In [ ]:
df1.info()

## Dataset 2

In [ ]:
df2.head()

In [ ]:
df2.shape

In [ ]:
df2.info()

## Dataset 3

In [ ]:
df3.head()

In [ ]:
df3.shape

In [ ]:
df3.info()

# Data Cleaning

From the cell above we see that not all of our datasets have the same consistency. There are some missing values, different currency values of house prices and different location naming systems. We need to clean this data to make it consistent.

## Cleaning df1

In [ ]:
# df1 has a few of missing values, so we will remove those rows with missing values
print('Shape of df1 before cleaning: ', df1.shape)
df1.dropna(inplace=True)
print('Shape of df1 after cleaning: ', df1.shape)

# the 'price_usd' column has $ infront of every values and commas, we need to drop these
df1['price_usd'] = df1['price_usd'].str.replace('$', '')
df1['price_usd'] = df1['price_usd'].str.replace(',', '')
df1.head()

## Cleaning df2

In [ ]:
# removing rows with missing values in df2
print('Shape of df2 before cleaning: ', df2.shape)
df2.dropna(inplace=True)
print('Shape of df2 after cleaning: ', df2.shape)

# df2 has the price in Mexican Pesos instead of US Dollars.
# 1USD = 19MXN
df2['price_usd'] = (df2['price_mxn'] / 19).round(2)

#dropping 'price_mxn' column
df2.drop(columns=['price_mxn'], inplace=True)
df2.head()

## Cleaning df3

In [ ]:
# removing rows with missing values in df3
print('Shape of df3 before cleaning: ', df3.shape)
df3.dropna(inplace=True)
print('Shape of df3 after cleaning: ', df3.shape)

# in df3, the naming system of the location of the property is different
# we will clean this to make the data consistent
df3['state'] = df3['place_with_parent_names'].str.split('|').str[2]

# dropping 'place_with_parent_names' column
df3.drop(columns= 'place_with_parent_names', inplace=True)

# breaking the 'lat-lon' column to 2 columns 'lat' and 'lon'
df3['lat'] = df3['lat-lon'].str.split(',').str[0]
df3['lon'] = df3['lat-lon'].str.split(',').str[1]

# dropping 'lat-lon' column
df3.drop(columns='lat-lon', inplace=True)

df3.head()

# Concatenate DataFrames


We are concetenating the 3 dataframes (df1, df2 and df3)to make df.

In [ ]:
# Concatenate df1, df2, and df3
df = pd.concat([df1, df2, df3])

# Print object type, shape, and head
print("df type:", type(df))
print("df shape:", df.shape)
df.head()

In [ ]:
  df["price_usd"] = df.price_usd.astype(float)

In [ ]:
df.info()

---

# Data Visualization

We are plotting the properties in a map. When we hover the mouse over each of them, the price will show up.

In [ ]:
# Use plotly express to create figure
fig = px.scatter_mapbox(
    df,  # Our DataFrame
    lat='lat',
    lon='lon',
    center={"lat": 19.43, "lon": -99.13},  # Map will be centered on Mexico City
    width=800,  # Width of map
    height=800,  # Height of map
    hover_data=["price_usd"],  # Display price when hovering mouse over house
)

# Add mapbox_style to figure layout
fig.update_layout(mapbox_style="open-street-map")

# Show figure
fig.show()

In [ ]:
df['state'].value_counts()

In [ ]:
df[["area_m2" , "price_usd"]].describe()

In [ ]:
plt.hist(df['area_m2'])
plt.xlabel('Area [sq meters]')
plt.ylabel('Frequency')
plt.title('Distribution of Home Sizes')
plt.show()

In [ ]:
# Use Matplotlib to create boxplot of "area_m2"
plt.boxplot(df["area_m2"], vert=False);

# Add x-axis label
plt.xlabel("Area [sq meters]")

# Add title
plt.title("Distribution of Home Sizes");

In [ ]:
# Use Matplotlib to create histogram of "price_usd"
plt.hist(df["price_usd"])

# Add x-axis label
plt.xlabel("Price [USD]")

# Add y-axis label
plt.ylabel("Frequency")

# Add title
plt.title("Distribution of Home Prices");

In [ ]:
# Use Matplotlib to create boxplot of "price_usd"
plt.boxplot(df["price_usd"], vert=False)

# Add x-label axis
plt.xlabel("Price [USD]")

# Add y-label axis
plt.title("Distribution of Home Prices");

## Analysis

In [ ]:
# Declare variable `mean_price_by_state`
mean_price_by_state = df.groupby("state")["price_usd"].mean().sort_values()

# Print object type, shape, and head
print("mean_price_by_state type:", type(mean_price_by_state))
print("mean_price_by_state shape:", mean_price_by_state.shape)
mean_price_by_state.head()

In [ ]:
# Create bar chart from `mean_price_by_state` using pandas
mean_price_by_state.plot(
    kind="bar",
    xlabel="State",
    ylabel="Price [USD]",
    title="Mean House Price by State"
);

It seems odd that Querétaro would be the most expensive real estate market in Mexico when, [according to recent GDP numbers](https://en.wikipedia.org/wiki/List_of_Mexican_states_by_GDP), it's not in the top 10 state economies. With all the variations in house sizes across states, a better metric to look at would be price per m<sup>2</sup>. In order to do that, we need to create a new column.

In [ ]:
# Create "price_per_m2" column
df["price_per_m2"] = df["price_usd"] / df["area_m2"]

# Print object type, shape, and head
print("df type:", type(df))
print("df shape:", df.shape)
df.head()

In [ ]:
# Group `df` by "state", create bar chart of "price_per_m2"
(
 df
    .groupby("state")["price_per_m2"]
    .mean()
    .sort_values(ascending=False)
    .plot(
    kind="bar",
    ylabel="Mean Price per m^2 [USD]",
    xlabel="State",
    title="Mean House Price per m^2 by State"
    )
);

Now we see that the capital Mexico City (*Distrito Federal*) is by far the most expensive market.  Additionally, many of the top 10 states by GDP are also in the top 10 most expensive real estate markets. So it looks like this bar chart is a more accurate reflection of state real estate markets.

From our previous question, we know that the location of a home affects its price (especially if it's in Mexico City), but what about home size? Does the size of a house influence price?

A scatter plot can be helpful when evaluating the relationship between two columns because it lets you see if two variables are correlated — in this case, if an increase in home size is associated with an increase in price.

In [ ]:
plt.scatter(x=df["area_m2"], y=df["price_usd"])
plt.xlabel("Area [sq meters]")
plt.ylabel("Price [USD]")
plt.title("Price vs Area");

In [ ]:
# Calculate correlation of "price_usd" and "area_m2"
p_correlation = df["area_m2"].corr(df["price_usd"])

# Print correlation coefficient
print("Correlation of 'area_m2' and 'price_usd' (all Mexico):", p_correlation)

## Subset of Morelos

The correlation coefficient is over 0.5, so there's a moderate relationship house size and price in Mexico. But does this relationship hold true in every state? Let's look at a couple of states, starting with Morelos.

In [ ]:
# Declare variable `df_morelos` by subsetting `df`
df_morelos = df[df["state"] == "Morelos"]

# Print object type, shape, and head
print("df_morelos type:", type(df_morelos))
print("df_morelos shape:", df_morelos.shape)
df_morelos.head()

## Scatter Plot of Properties in Morelos

In [ ]:
plt.scatter(x=df_morelos["area_m2"], y=df_morelos["price_usd"])
plt.xlabel("Area [sq meters]")
plt.ylabel("Price [USD]")
plt.title("Morelos Price vs Area")

In [ ]:
# Calculate correlation of "price_usd" and "area_m2" in `df_morelos`
p_correlation = df_morelos["area_m2"].corr(df_morelos["price_usd"])

# Print correlation coefficient
print("Correlation of 'area_m2' and 'price_usd' (Morelos):", p_correlation)

With a correlation coefficient that high, we can say that there's a strong relationship between house size and price in Morelos.

To conclude, let's look at the capital Mexico City (*Distrito Federal*).  

In [ ]:
df_mexico_city = df[df["state"] == "Distrito Federal"]

print("df_mexico_city type:", type(df_mexico_city))
print("df_mexico_city shape:", df_mexico_city.shape)

plt.scatter(df_mexico_city["area_m2"], df_mexico_city["price_usd"])
plt.xlabel("Area [sq meters]")
plt.ylabel("Price [USD]")
plt.title("Mexico City: Price vs. Area");

p_correlation = df_morelos["area_m2"].corr(df_morelos["price_usd"])
print("Correlation of 'area_m2' and 'price_usd' (Mexico City):", p_correlation)

Looking at the scatter plot and correlation coefficient, there's see a weak relationship between size and price. How should we interpret this?

One interpretation is that the relationship we see between size and price in many states doesn't hold true in the country's biggest and most economically powerful urban center because there are other factors that have a larger influence on price. In fact, in the next project, we're going to look at another important Latin American city — Buenos Aires, Argentina — and build a model that predicts housing price by taking much more than size into account.

# Training Model